In [2]:
# Coneccion entre Google Colab y el Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **<font color="#ff5653">Funciones y Machine Learning</font>**

1- PlayTimeGenre

2- UserForGenre

3- UsersRecommend

4- UsersNotRecommend

5- Sentiment_analysis

6- Game_Recommendation

In [1]:
# Libreria
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Cargamos dataframe df_f1yf2
df_f1yf2 = pd.read_parquet("/content/drive/MyDrive/Datasets_Steam/df_f1yf2.parquet")

In [5]:
# Cargamos dataframe df_f3f4yf5
df_f3f4yf5 = pd.read_parquet("/content/drive/MyDrive/Datasets_Steam/df_f3f4yf5.parquet")

In [6]:
# Cargamos dataframe df_fml1
df_f6_ml = pd.read_parquet("/content/drive/MyDrive/Datasets_Steam/df_f6_ml.parquet")

### 1- **def PlayTimeGenre**( genero : str ):

- Debe devolver año con mas horas jugadas para dicho género.

- Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

- Dataframe a utilizar: df_f1yf2

- Columnas a utilizar: genres, año_lanzamiento y playtime_forever.

In [ ]:
# Extraemos las columnas a utilizar
df_1 = df_f1yf2[["genres", "año_lanzamiento", "playtime_forever"]]

In [ ]:
# Generos
unique_genres = set()
for genres_list in df_1['genres']:
    unique_genres.update(genres_list)

In [ ]:
# Lista de genero
unique_genres_list = list(unique_genres)

In [ ]:
unique_genres

{'Action',
 'Adventure',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

In [ ]:
# Pasamos todoas las palabras del genero a mayusculas
def normalize_list_of_words(sentences_list):

    normalized_sentences = []

    for sentence in sentences_list:
        words = sentence.split()
        normalized_words = [word.capitalize() for word in words]
        normalized_sentence = ' '.join(normalized_words)
        normalized_sentences.append(normalized_sentence)

    return normalized_sentences

In [ ]:
normalized_genres = normalize_list_of_words(unique_genres_list)

In [ ]:
def capitalize_first_words_in_sentence(sentence):

    words = sentence.split()
    capitalized_words = [word[0].capitalize() + word.lower()[1:] for word in words]
    return ' '.join(capitalized_words)

In [ ]:
def PlayTimeGenre(genre):

    if type(genre) == str:
        norm_genre = capitalize_first_words_in_sentence(genre)

        if norm_genre in normalized_genres:
            genre_to_find = norm_genre

            if norm_genre == 'Free To Play':
                genre_to_find = 'Free to Play'
            elif norm_genre == 'Rpg':
                genre_to_find = 'RPG'


            mask = df_1['genres'].apply(lambda x: genre_to_find in x)


            df_f1_by_genre = df_1[mask]


            grouped_df_f1_by_year = df_f1_by_genre.groupby('año_lanzamiento')['playtime_forever'].sum().reset_index()


            df1genre = grouped_df_f1_by_year[['año_lanzamiento','playtime_forever']]


            df1genre.sort_values(by='playtime_forever', ascending=False, inplace=True)


            df1genre.reset_index(drop=True, inplace=True)

            year_most_hours_played = df1genre.iloc[0,0]
            max_sum_playtime_forever = df1genre.iloc[0,1]


            F1message = {
                f"Anio de lanzamiento con mas horas reproducidas para Genero {genre_to_find}": int(year_most_hours_played)
            }

            json_message = json.dumps(F1message, indent=4)

            return print(json_message)

        else:
            return print("Inserte un ganero valido.")
    else:
        return print("Inserte un ganero valido.")

In [ ]:
PlayTimeGenre("Action")

{
    "Anio de lanzamiento con mas horas reproducidas para Genero Action": 2012
}


In [ ]:
PlayTimeGenre("Adventure")

{
    "Anio de lanzamiento con mas horas reproducidas para Genero Adventure": 2013
}


In [ ]:
PlayTimeGenre("fsasasa")

Inserte un ganero valido.


In [ ]:
PlayTimeGenre(345)

Inserte un ganero valido.


### 2- **def UserForGenre**( genero : str ):

- Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

- Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

- Dataframe a utilizar: df_f1yf2

- Columnas a utilizar: user_id, genres, año_lanzamiento y playtime_forever.

In [ ]:
df_2 = df_f1yf2[["user_id","genres","año_lanzamiento","playtime_forever"]]

In [ ]:
def UseForGenre(genre):

    if type(genre) == str:

        norm_genre = capitalize_first_words_in_sentence(genre)

        if norm_genre in normalized_genres:

            genre_to_find = norm_genre

            if norm_genre == 'Free To Play':
                genre_to_find = 'Free to Play'
            elif norm_genre == 'Rpg':
                genre_to_find = 'RPG'

            mask = df_2['genres'].apply(lambda x: genre_to_find in x)

            df_f2_by_genre = df_2[mask]

            grouped_df_f2_by_user_year = df_f2_by_genre.groupby(['user_id','año_lanzamiento'])['playtime_forever'].sum().reset_index()

            grouped_df_f2_by_user = df_f2_by_genre.groupby(['user_id'])['playtime_forever'].sum().reset_index()

            grouped_df_f2_by_user.sort_values(by='playtime_forever', ascending=False, inplace=True)

            grouped_df_f2_by_user.reset_index(drop=True, inplace=True)

            user_most_hours_played = grouped_df_f2_by_user.iloc[0,0]

            mask = grouped_df_f2_by_user_year['user_id'] == user_most_hours_played
            resultF2 = grouped_df_f2_by_user_year[mask]

            playtime_list = resultF2.rename(columns={'año_lanzamiento': 'Release Year', 'playtime_forever': 'Hours'})[['Release Year', 'Hours']].to_dict(orient='records')

            F2message = {
                f"Usuario con mas horas reproducidas por Genero {genre_to_find}": user_most_hours_played,
                "Tiempo de juego": playtime_list
            }

            json_message = json.dumps(F2message, indent=4)

            return print(json_message)
        else:
            return print("Inserte un ganero valido")
    else:
        return print("Inserte un ganero valido")

In [ ]:
UseForGenre("Action")

{
    "Usuario con mas horas reproducidas por Genero Action": "REBAS_AS_F-T",
    "Tiempo de juego": [
        {
            "Release Year": 1998,
            "Hours": 2.0
        },
        {
            "Release Year": 1999,
            "Hours": 227.0
        },
        {
            "Release Year": 2001,
            "Hours": 33.0
        },
        {
            "Release Year": 2002,
            "Hours": 1.0
        },
        {
            "Release Year": 2003,
            "Hours": 5589.0
        },
        {
            "Release Year": 2004,
            "Hours": 9102.0
        },
        {
            "Release Year": 2005,
            "Hours": 5519.0
        },
        {
            "Release Year": 2006,
            "Hours": 3556.0
        },
        {
            "Release Year": 2007,
            "Hours": 11585.0
        },
        {
            "Release Year": 2008,
            "Hours": 9663.0
        },
        {
            "Release Year": 2009,
            "Hours": 118276.0
 

### 3- **def UsersRecommend**( año : int ):

- Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

- Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

- Dataframe a utilizar: df_f3f4yf5

- Columnas a utilizar: item_id, title, recommend, analisis_sentimiento y año_publicado.


In [ ]:
df_3 = df_f3f4yf5[["item_id","title","recommend","analisis_sentimiento","año_publicado"]]

In [ ]:
def UsersRecommend(year):

    if type(year) == int:

        mask = df_3['año_publicado'] == year
        df_f3_review_year = df_3[mask].reset_index(drop=True)

        grouped_df_f3_review_year = df_f3_review_year.groupby(['title'])['analisis_sentimiento'].sum().reset_index()

        grouped_df_f3_review_year.sort_values(by='analisis_sentimiento', ascending=False, inplace=True)

        if not grouped_df_f3_review_year.empty:

            item_Rank_1 = grouped_df_f3_review_year.iloc[0,0]

            item_Rank_2 = grouped_df_f3_review_year.iloc[1,0]

            item_Rank_3 = grouped_df_f3_review_year.iloc[2,0]

            dataF3 = {
                'Rank': ['Position 1', 'Position 2', 'Position 3'],
                'title': [item_Rank_1, item_Rank_2, item_Rank_3]
            }

            json_data = [{"Rank " + str(index + 1): item} for index, item in enumerate(dataF3['title'])]

            json_message = json.dumps(json_data, indent=None, ensure_ascii=False)

            return print(json_message)
        else:
            return print("No hay reseñas para el anio ingresado")
    else:
        return print("No hay reseñas para el anio ingresado")

In [ ]:
UsersRecommend(2000)

No hay reseñas para el anio ingresado


In [ ]:
UsersRecommend(2012)

[{"Rank 1": "Garry's Mod"}, {"Rank 2": "Terraria"}, {"Rank 3": "Left 4 Dead 2"}]


### 4- **def UsersNotRecommend**( año : int ):

- Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

- Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

- Dataframe a utilizar: df_f3f4yf5

- Columnas a utilizar: item_id, title, recommend, analisis_sentimiento y año_publicado.

In [ ]:
df_4 = df_f3f4yf5[["item_id","title","recommend","analisis_sentimiento","año_publicado"]]

In [ ]:
def UsersNotRecommend(year):

    if type(year) == int:

        mask = df_4['año_publicado'] == year
        df_f4_review_year = df_4[mask].reset_index(drop=True)

        if not df_f4_review_year.empty:

            counter = df_f4_review_year['title'].value_counts()

            top_3_least_recommended = counter.head(3)

            json_data = [{"Rank " + str(index + 1): item} for index, item in enumerate(top_3_least_recommended.index)]

            json_message = json.dumps(json_data, indent=None, ensure_ascii=False)

            return print(json_message)
        else:
            return print("No hay reseñas para el anio ingresado.")
    else:
        return print("No hay reseñas para el anio ingresado.")

In [ ]:
UsersNotRecommend(2000)

No hay reseñas para el anio ingresado.


In [ ]:
UsersNotRecommend(2016)

[{"Rank 1": "Counter-Strike: Global Offensive"}, {"Rank 2": "Garry's Mod"}, {"Rank 3": "Unturned"}]


### 5- **def sentiment_analysis**( año : int ):

- Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

- Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

- Dataframe a utilizar: df_f3f4yf5

- Columnas a utilizar: analisis_sentimiento y año_lanzamiento.

In [ ]:
df_5 = df_f3f4yf5[["analisis_sentimiento","año_lanzamiento"]]

In [ ]:
def Sentiment_analysis(year):

    if type(year) == int:

        mask = df_5['año_lanzamiento'] == year
        df_f5_review_year = df_5[mask].reset_index(drop=True)

        if not df_f5_review_year.empty:

            counter = df_f5_review_year['analisis_sentimiento'].value_counts().sort_index()

            dataF5 = {
                "Negativo": int(counter.get(0, 0)),
                "Neutral": int(counter.get(1, 0)),
                "Positivo": int(counter.get(2, 0))
            }

            json_message = json.dumps(dataF5, indent=None, ensure_ascii=False)

            return print(json_message)
        else:
            return print("El anio ingresado no tiene registro para realizar el análisis de sentimiento.")
    else:
        return print("El anio ingresado no tiene registro para realizar el análisis de sentimiento.")

In [ ]:
Sentiment_analysis(2015)

{"Negativo": 900, "Neutral": 979, "Positivo": 3693}


In [ ]:
Sentiment_analysis(2000)

{"Negativo": 7, "Neutral": 16, "Positivo": 59}


In [ ]:
Sentiment_analysis(2018)

El anio ingresado no tiene registro para realizar el análisis de sentimiento.


### 6- **def Game_Recommendation:**( id_producto: int ):

- Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

- Ejemplo de retorno: Ingresamos > 761140

   - **<font color="#80BFFF">Game_Recommendation</font>**(761140)

   - [{'Recommendation 1': 'World of Cinema - Directors Cut'},
   - {'Recommendation 2': 'Aerial Destruction'},
   - {'Recommendation 3': 'Cyber Complex'},
   - {'Recommendation 4': "Trivia Vault: 1980's Trivia 2"},
   - {'Recommendation 5': 'Trivia Vault: Science &amp; History Trivia 2'}]

- Dataframe a utilizar: df_f6_ml

- Columnas a utilizar: "item_id" y "title"

In [7]:
df_f6_ml = pd.read_parquet("C:/Users/hpiza/Desktop/Subir a GitHub/Proyecto_individualDS-1/3_Datasets/df_f6_ml.parquet")

In [8]:
df_FML1_for_Model = df_f6_ml

In [ ]:
# Combina columnas de texto en una sola columna para vectorización
df_FML1_for_Model['combined_features'] = df_FML1_for_Model['tags']

# Completar los valores faltantes
df_FML1_for_Model['combined_features'].fillna('', inplace=True)

In [18]:
# Convertimos de texto a vectores numericos
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_FML1_for_Model['combined_features'])

In [11]:
# Calculamos la similitud del coseno
# Usamos la función cosine_similarity de la libreria scikit-learn.
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# En Google Colab, se me acabaron los ram gratis y en VSC me tardo 25 minutos.

In [51]:
# Funcion
def Game_Recommendation(item_id, cosine_sim=cosine_sim):
# Al ingresar el `item_id`, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.
    
    # Revisamos que el usuario ingrese un número
    if (type(item_id) == float) | (type(item_id) == int):
        
        # Ahora, revisemos la item_id existe
        if item_id in df_FML1_for_Model['item_id'].values:
            # Capturamos el índice del juego en la columna 'item_id'
            game_index = df_FML1_for_Model[df_FML1_for_Model['item_id'] == item_id].index[0]
            # Calculamos puntuaciones de similitud de cosenos para el juego de entrada con todos los demás juegos.
            similar_scores = list(enumerate(cosine_sim[game_index]))
            # Ordenamos los puntajes_similares según los valores de similitud del coseno en orden descendente
            similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)
            # Excluimos el juego de entrada en sí (el juego más similar será el juego de entrada)
            similar_scores = similar_scores[1:6]

            # Creamos una lista de juegos recomendados en formato JSON.
            recommended_games = [{"Recommendation {}".format(i + 1): df_FML1_for_Model.iloc[score[0]]['title']} for i, score in enumerate(similar_scores)]

            return recommended_games
        
        else:
            return print("El item_id no existe. Inténtalo de nuevo.")
    else:
        return print("El item_id debe ser un número; ingrese un nuevo item_id")

In [25]:
# Vemos columna "item_id", donde esta la clave de cada "title".
df_FML1_for_Model

,item_id,title,tags,combined_features
0,761140.0,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...","['Strategy', 'Action', 'Indie', 'Casual', 'Sim..."
1,643980.0,Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Free to Play', 'Strategy', 'Indie', 'RPG', '..."
2,670290.0,Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Free to Play', 'Simulation', 'Sports', 'Casu..."
3,767400.0,弹炸人2222,"['Action', 'Adventure', 'Casual']","['Action', 'Adventure', 'Casual']"
4,772540.0,Battle Royale Trainer,"['Action', 'Adventure', 'Simulation', 'FPS', '...","['Action', 'Adventure', 'Simulation', 'FPS', '..."
...,...,...,...,...
29917,745400.0,Kebab it Up!,"['Action', 'Indie', 'Casual', 'Violent', 'Adve...","['Action', 'Indie', 'Casual', 'Violent', 'Adve..."
29918,773640.0,Colony On Mars,"['Strategy', 'Indie', 'Casual', 'Simulation']","['Strategy', 'Indie', 'Casual', 'Simulation']"
29919,733530.0,LOGistICAL: South Africa,"['Strategy', 'Indie', 'Casual']","['Strategy', 'Indie', 'Casual']"
29920,610660.0,Russian Roads,"['Indie', 'Simulation', 'Racing']","['Indie', 'Simulation', 'Racing']"


In [52]:
Game_Recommendation(761140)

[{'Recommendation 1': 'World of Cinema - Directors Cut'},
 {'Recommendation 2': 'Aerial Destruction'},
 {'Recommendation 3': 'Cyber Complex'},
 {'Recommendation 4': "Trivia Vault: 1980's Trivia 2"},
 {'Recommendation 5': 'Trivia Vault: Science &amp; History Trivia 2'}]

In [53]:
Game_Recommendation(767400.0)

[{'Recommendation 1': 'Atomic Adam: Episode 1'},
 {'Recommendation 2': 'Biozone'},
 {'Recommendation 3': 'Abandoned Knight'},
 {'Recommendation 4': 'Super Happy Singh'},
 {'Recommendation 5': 'Luke Sidewalker'}]

In [54]:
Game_Recommendation(733530.0)

[{'Recommendation 1': 'Ticket to Ride - USA 1910'},
 {'Recommendation 2': 'Ticket to Ride - Europe'},
 {'Recommendation 3': 'Tribloos 2'},
 {'Recommendation 4': 'Hive - The Ladybug'},
 {'Recommendation 5': 'Bounders and Cads'}]

In [55]:
Game_Recommendation(610660.0)

[{'Recommendation 1': 'Car Mechanic Simulator 2015 - Total Modifications'},
 {'Recommendation 2': 'SoundTracks: The Train Set Game'},
 {'Recommendation 3': 'RC-AirSim - RC Model Airplane Flight Simulator'},
 {'Recommendation 4': 'Car Mechanic Simulator 2015 - DeLorean'},
 {'Recommendation 5': 'Russian Roads'}]

In [46]:
Game_Recommendation(200)

El item_id no existe. Inténtalo de nuevo.


In [47]:
Game_Recommendation("item")

El item_id debe ser un número; ingrese un nuevo item_id
